In [ ]:
import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler,MinMaxScaler
X_ppg,X_qual,X_acc,y_rr, y_respiration,y_inspiration,y_expiration,groups,X_respiration = pickle.load(open('../../affect-recognition/data/tabular_data.p','rb'))

In [ ]:
from scipy import signal
X_ppg = X_ppg.reshape(-1,512,1)
X_ppg = np.array([MinMaxScaler().fit_transform(signal.resample(a.reshape(-1),256).reshape(-1,1)) for a in X_ppg])
print(X_ppg.shape,X_acc.shape)

In [1]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# load fashion mnist images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return [X, trainy]

# # select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
    g_model.save('cgan_generator.h5')

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/home/azim/.local/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/azim/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/azim/.local/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/azim/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWar

>1, 1/468, d1=0.689, d2=0.696 g=0.692


/home/azim/.local/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 2/468, d1=0.593, d2=0.697 g=0.691
>1, 3/468, d1=0.514, d2=0.700 g=0.688
>1, 4/468, d1=0.443, d2=0.710 g=0.677
>1, 5/468, d1=0.374, d2=0.731 g=0.657
>1, 6/468, d1=0.313, d2=0.766 g=0.626
>1, 7/468, d1=0.263, d2=0.828 g=0.587
>1, 8/468, d1=0.226, d2=0.895 g=0.546
>1, 9/468, d1=0.210, d2=0.963 g=0.511
>1, 10/468, d1=0.221, d2=1.016 g=0.512
>1, 11/468, d1=0.242, d2=1.003 g=0.541
>1, 12/468, d1=0.261, d2=0.909 g=0.593
>1, 13/468, d1=0.304, d2=0.832 g=0.674
>1, 14/468, d1=0.345, d2=0.720 g=0.768
>1, 15/468, d1=0.361, d2=0.644 g=0.868
>1, 16/468, d1=0.371, d2=0.556 g=0.956
>1, 17/468, d1=0.416, d2=0.504 g=1.069
>1, 18/468, d1=0.382, d2=0.435 g=1.170
>1, 19/468, d1=0.361, d2=0.394 g=1.253
>1, 20/468, d1=0.356, d2=0.345 g=1.378
>1, 21/468, d1=0.445, d2=0.331 g=1.400
>1, 22/468, d1=0.375, d2=0.312 g=1.480
>1, 23/468, d1=0.302, d2=0.282 g=1.559
>1, 24/468, d1=0.272, d2=0.255 g=1.663
>1, 25/468, d1=0.297, d2=0.243 g=1.744
>1, 26/468, d1=0.274, d2=0.200 g=1.858
>1, 27/468, d1=0.208, d2=0.191 g=

>1, 210/468, d1=0.000, d2=0.003 g=6.496
>1, 211/468, d1=0.001, d2=0.002 g=6.435
>1, 212/468, d1=0.002, d2=0.002 g=6.423
>1, 213/468, d1=0.005, d2=0.002 g=6.323
>1, 214/468, d1=0.001, d2=0.003 g=6.458
>1, 215/468, d1=0.000, d2=0.002 g=6.510
>1, 216/468, d1=0.001, d2=0.002 g=6.556
>1, 217/468, d1=0.000, d2=0.002 g=6.760
>1, 218/468, d1=0.001, d2=0.001 g=6.651
>1, 219/468, d1=0.002, d2=0.002 g=6.675
>1, 220/468, d1=0.001, d2=0.002 g=6.706
>1, 221/468, d1=0.001, d2=0.002 g=6.762
>1, 222/468, d1=0.001, d2=0.002 g=6.772
>1, 223/468, d1=0.001, d2=0.002 g=6.718
>1, 224/468, d1=0.003, d2=0.002 g=6.772
>1, 225/468, d1=0.001, d2=0.001 g=6.794
>1, 226/468, d1=0.003, d2=0.002 g=6.791
>1, 227/468, d1=0.000, d2=0.001 g=6.754
>1, 228/468, d1=0.001, d2=0.002 g=6.883
>1, 229/468, d1=0.000, d2=0.001 g=7.002
>1, 230/468, d1=0.000, d2=0.002 g=6.851
>1, 231/468, d1=0.003, d2=0.001 g=6.941
>1, 232/468, d1=0.000, d2=0.001 g=6.925
>1, 233/468, d1=0.000, d2=0.001 g=6.980
>1, 234/468, d1=0.007, d2=0.001 g=6.745


>1, 415/468, d1=0.000, d2=0.000 g=8.036
>1, 416/468, d1=0.000, d2=0.000 g=8.170
>1, 417/468, d1=0.000, d2=0.000 g=8.208
>1, 418/468, d1=0.000, d2=0.000 g=8.111
>1, 419/468, d1=0.000, d2=0.000 g=8.311
>1, 420/468, d1=0.000, d2=0.000 g=8.326
>1, 421/468, d1=0.001, d2=0.000 g=8.119
>1, 422/468, d1=0.001, d2=0.000 g=8.025
>1, 423/468, d1=0.000, d2=0.000 g=8.157
>1, 424/468, d1=0.000, d2=0.000 g=8.332
>1, 425/468, d1=0.000, d2=0.000 g=8.200
>1, 426/468, d1=0.000, d2=0.000 g=8.373
>1, 427/468, d1=0.000, d2=0.000 g=8.484
>1, 428/468, d1=0.000, d2=0.000 g=8.330
>1, 429/468, d1=0.000, d2=0.000 g=8.386
>1, 430/468, d1=0.000, d2=0.000 g=8.381
>1, 431/468, d1=0.000, d2=0.000 g=8.331
>1, 432/468, d1=0.000, d2=0.000 g=8.451
>1, 433/468, d1=0.001, d2=0.000 g=8.307
>1, 434/468, d1=0.002, d2=0.000 g=8.191
>1, 435/468, d1=0.013, d2=0.001 g=7.260
>1, 436/468, d1=0.000, d2=0.002 g=7.165
>1, 437/468, d1=0.000, d2=0.001 g=7.283
>1, 438/468, d1=0.000, d2=0.001 g=7.552
>1, 439/468, d1=0.000, d2=0.001 g=7.736


KeyboardInterrupt: 